In [1]:
# Propagation from AZ/EL observations
# Perturbation:
# N/A
# Maneuver: 
# No

# IOD
# Given from propagation_issX

# Obserbation:
# AZ/EL
# TOD / IERS_2010

# Output:
# TBD


In [2]:
%matplotlib inline

from math import radians, pi
import pandas as pd
import numpy as np
import plotly.express as px


import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()


from org.orekit.orbits import KeplerianOrbit, PositionAngle
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate, TimeScalesFactory, TimeScale
from org.orekit.utils import Constants
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import LofOffset
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.forces.drag import IsotropicDrag
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.frames import FramesFactory, LOFType, EOPHistory
from org.orekit.models.earth.atmosphere import HarrisPriester
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.propagation import PropagationType, SpacecraftState
from org.orekit.propagation.events import DateDetector, EventEnablingPredicateFilter, PythonEnablingPredicate,\
     PositionAngleDetector, PythonEventDetector, AbstractDetector, EventDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder, \
                        ClassicalRungeKuttaIntegratorBuilder, EulerIntegratorBuilder
from org.orekit.propagation.semianalytical.dsst.forces import DSSTAtmosphericDrag, DSSTNewtonianAttraction, DSSTZonal
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from orekit.pyhelpers import absolutedate_to_datetime
from org.hipparchus.linear import RealMatrix
from org.hipparchus.linear import QRDecomposer, AbstractRealMatrix
from org.hipparchus.optim.nonlinear.vector.leastsquares import GaussNewtonOptimizer
from org.orekit.estimation.leastsquares import BatchLSEstimator
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.propagation.conversion import AbstractPropagatorBuilder

from org.orekit.estimation.measurements import AngularAzEl
from org.orekit.estimation.measurements import ObservableSatellite
from org.orekit.estimation.measurements import GroundStation
from org.orekit.estimation.sequential import KalmanEstimatorBuilder
from org.orekit.estimation.sequential import KalmanEstimator
from org.orekit.estimation.sequential import KalmanEstimation
from org.orekit.estimation.measurements import *
from org.orekit.estimation.iod import IodGooding, IodLaplace

from orekit.pyhelpers import absolutedate_to_datetime, datetime_to_absolutedate
from collections import namedtuple
from org.orekit.time import Month
from orekit import JArray

from org.orekit.estimation.sequential import KalmanEstimator, KalmanEstimatorBuilder, CovarianceMatrixProvider, ConstantProcessNoise
from org.orekit.orbits import Orbit


In [3]:
utc = TimeScalesFactory.getUTC()

# Given by ADS
a = 6000000.0  
e = 0.5
i = radians(98.0)     
omega = radians(0.2)   # perigee argument
raan = radians(90.0)  # right ascension of ascending node
lv = radians(0.0)#0.1)    # True anomaly

#mass = 625.0

epochDate = AbsoluteDate(2020, 1, 1, 0, 0, 00.000, utc)
initialDate = epochDate

## Inertial frame where the satellite is defined
inertialFrame =  FramesFactory.getTOD(IERSConventions.IERS_2010, False)
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         inertialFrame)

## Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lv,
                              PositionAngle.TRUE,
                              inertialFrame, epochDate, Constants.WGS84_EARTH_MU)

utc = TimeScalesFactory.getUTC()

# Orbit propagator parameters
prop_min_step = 60.0 # s
prop_max_step = 300.0 # s
prop_position_error = 100.0 # m

# Estimator parameters
estimator_position_scale = 1.0 # m
estimator_convergence_thres = 1e-3
estimator_max_iterations = 25
estimator_max_evaluations = 35

moon = CelestialBodyFactory.getMoon()
sun = CelestialBodyFactory.getSun()

In [4]:
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder
integratorBuilder = DormandPrince853IntegratorBuilder(prop_min_step, prop_max_step, prop_position_error)
#integratorBuilder = EulerIntegratorBuilder(10.0)

from org.orekit.propagation.conversion import NumericalPropagatorBuilder
from org.orekit.orbits import PositionAngle
propagatorBuilder = NumericalPropagatorBuilder(initialOrbit,
                                               integratorBuilder, PositionAngle.MEAN, estimator_position_scale)
#propagatorBuilder.setMass(1000.0)
#propagatorBuilder.setAttitudeProvider(nadirPointing)

In [5]:
# Earth gravity field with degree 64 and order 64
from org.orekit.forces.gravity.potential import GravityFieldFactory
gravityProvider = GravityFieldFactory.getConstantNormalizedProvider(20,20)
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
gravityAttractionModel = HolmesFeatherstoneAttractionModel(inertialFrame, gravityProvider)
propagatorBuilder.addForceModel(gravityAttractionModel)

# Moon and Sun perturbations
#from org.orekit.forces.gravity import ThirdBodyAttraction
#moon_3dbodyattraction = ThirdBodyAttraction(moon)
#propagatorBuilder.addForceModel(moon_3dbodyattraction)
#sun_3dbodyattraction = ThirdBodyAttraction(sun)
#propagatorBuilder.addForceModel(sun_3dbodyattraction)

# Solar radiation pressure
#from org.orekit.forces.radiation import IsotropicRadiationSingleCoefficient
#isotropicRadiationSingleCoeff = IsotropicRadiationSingleCoefficient(sat_list[sc_name]['cross_section'], sat_list[sc_name]['cr']);
#from org.orekit.forces.radiation import SolarRadiationPressure
#solarRadiationPressure = SolarRadiationPressure(sun, wgs84Ellipsoid.getEquatorialRadius(),
#                                                isotropicRadiationSingleCoeff)
#propagatorBuilder.addForceModel(solarRadiationPressure)

# Atmospheric drag
#from org.orekit.forces.drag.atmosphere.data import MarshallSolarActivityFutureEstimation
#msafe = MarshallSolarActivityFutureEstimation(
#    '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\p{Digit}\p{Digit}\p{Digit}\p{Digit}F10\.(?:txt|TXT)',
#    MarshallSolarActivityFutureEstimation.StrengthLevel.AVERAGE)
#DM.feed(msafe.getSupportedNames(), msafe) # Feeding the F10.7 bulletins to Orekit's data manager

#from org.orekit.forces.drag.atmosphere import NRLMSISE00
#atmosphere = NRLMSISE00(msafe, sun, wgs84Ellipsoid)
#from org.orekit.forces.drag.atmosphere import DTM2000
#atmosphere = DTM2000(msafe, sun, wgs84Ellipsoid)
#from org.orekit.forces.drag import IsotropicDrag
#isotropicDrag = IsotropicDrag(sat_list[sc_name]['cross_section'], sat_list[sc_name]['cd'])
#from org.orekit.forces.drag import DragForce
#dragForce = DragForce(atmosphere, isotropicDrag)
#propagatorBuilder.addForceModel(dragForce)

# Relativity
#from org.orekit.forces.gravity import Relativity
#relativity = Relativity(orekit_constants.EIGEN5C_EARTH_MU)
#propagatorBuilder.addForceModel(relativity)

In [6]:
matrixDecomposer = QRDecomposer(1e-11)
optimizer = GaussNewtonOptimizer(matrixDecomposer, False)

estimator = BatchLSEstimator(optimizer, propagatorBuilder)
estimator.setParametersConvergenceThreshold(estimator_convergence_thres)
estimator.setMaxIterations(estimator_max_iterations)
estimator.setMaxEvaluations(estimator_max_evaluations)

In [7]:
# Fetching range data

#Open and collect data from csv
file = pd.read_csv("export_all_NoPert.csv")
az = file['azimuth']
el = file['elevation']
datetime = file['pv']
idx = datetime[0].find(',')

errDeg = 0.001 #10*0.000277778 #1 arcsec
# TBD
azBaseWeight = 1.0
elBaseWeight = 1.0

# Load values
# Ground station creation
#frame = FramesFactory.getTOD(False)
#frame = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
frame = FramesFactory.getTOD(IERSConventions.IERS_2010, False)

earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         frame)
longitude = radians(21.038)
latitude  = radians(67.8790708)
altitude  = 527.0
station = GeodeticPoint(latitude, longitude, altitude)
stationFrame = TopocentricFrame(earth, station, "Kiruna")


ObsSat= ObservableSatellite(0) # From R5
for j in range(0,100):
    year = int(datetime[j][1:5])
    month = int(datetime[j][6:8])
    day = int(datetime[j][9:11])
    hour = int(datetime[j][12:14])
    minute = int(datetime[j][15:17])
    second = int(datetime[j][18:20])
    date = AbsoluteDate(year, Month.getMonth(month), day, hour, minute, 0.0, utc)#TimeScalesFactory.getUT1(IERSConventions.IERS_2010, False))
    orekitAzEl = AngularAzEl(GroundStation(stationFrame),
					#datetime_to_absolutedate(datetime[j][1:idx]),
                    date,
					JArray('double')([radians(az[j]),radians(el[j])]),
                    JArray('double')([radians(errDeg),radians(errDeg)]),
                    JArray('double')([azBaseWeight,elBaseWeight]),
                    ObsSat)
    estimator.addMeasurement(orekitAzEl)



In [8]:
# Orbit estimation
estimatedPropagatorArray = estimator.estimate()

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
org.orekit.errors.OrekitIllegalArgumentException: orbit should be either elliptic with a > 0 and e < 1 or hyperbolic with a < 0 and e > 1, a = -2,486,285.656, e = -1.6
	at org.orekit.orbits.KeplerianOrbit.<init>(KeplerianOrbit.java:201)
	at org.orekit.orbits.KeplerianOrbit.<init>(KeplerianOrbit.java:163)
	at org.orekit.orbits.OrbitType$4.mapArrayToOrbit(OrbitType.java:492)
	at org.orekit.orbits.OrbitType$4.mapArrayToOrbit(OrbitType.java:448)
	at org.orekit.propagation.numerical.NumericalPropagator$OsculatingMapper.mapArrayToState(NumericalPropagator.java:435)
	at org.orekit.propagation.integration.StateMapper.mapArrayToState(StateMapper.java:167)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator$ConvertedMainStateEquations.computeDerivatives(AbstractIntegratedPropagator.java:674)
	at org.hipparchus.ode.ExpandableODE.computeDerivatives(ExpandableODE.java:134)
	at org.hipparchus.ode.Abstrac

In [ ]:
estimatedPropagator = estimatedPropagatorArray[0]
estimatedInitialState = estimatedPropagator.getInitialState()
actualOdDate = estimatedInitialState.getDate()
estimatedOrbit_init = estimatedInitialState.getOrbit()

In [ ]:
print(estimatedPropagator)
print(estimatedInitialState)
print(actualOdDate)
print(estimatedOrbit_init)
print(estimatedPropagator.getEventsDetectors())